In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14
from scipy.stats import linregress
from scipy.interpolate import RegularGridInterpolator
from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator,  FormatStrFormatter

from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_p(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["p1"][particle, :][mask],
        track.data["p2"][particle, :][mask],
        track.data["p3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj


def get_fields(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["E1"][particle, :][mask],
        track.data["E2"][particle, :][mask],
        track.data["E3"][particle, :][mask],
        track.data["B1"][particle, :][mask],
        track.data["B2"][particle, :][mask],
        track.data["B3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_gradB(sim, t=None):
    
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    dB_x1 = ou.Derivative_Diagnostic(B, "x1")
    dB_x2 = ou.Derivative_Diagnostic(B, "x2")
    dB_x3 = ou.Derivative_Diagnostic(B, "x3")
    
    if isinstance(t, int):
        return np.stack([
            dB_x1[t],
            dB_x2[t],
            dB_x3[t]
        ], axis=0)
    else:
        return np.stack([
            dB_x1,
            dB_x2,
            dB_x3
        ], axis=0)

def get_b(sim, t=None):
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    b1 = sim_centered["part_b1"] / B
    b2 = sim_centered["part_b2"] / B
    b3 = sim_centered["part_b3"] / B

    b = np.stack([b1, b2, b3], axis=0)

    if isinstance(t, int):
        return b[t]
    else:
        return b
    
def get_Jacobian_b(sim, t=None):
    
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    b1 = sim_centered["part_b1"] / B
    b2 = sim_centered["part_b2"] / B
    b3 = sim_centered["part_b3"] / B

    J_B = np.empty((3, 3), dtype=object)
    for i, b_buffer in enumerate([b1, b2, b3]):
        for j, axis in enumerate(["x1", "x2", "x3"]):
            J_B[i,j] = ou.Derivative_Diagnostic(b_buffer, axis)

    if isinstance(t, int):
        return J_B[:, :, t]
    else:
        return J_B


def get_Jacobian_b_Interp(J_B, t=None):
    vfunc = np.vectorize(diag_interpolator, otypes=[object])
    return vfunc(J_B, t=t)


def diag_interpolator(diag, t=0):
    # TODO maybe considder periodic boundaries
    
    # Define grid points
    x = np.linspace(diag.grid[0][0] + diag.dx[0] / 2., diag.grid[0][1] - diag.dx[0] / 2., diag.nx[0])
    y = np.linspace(diag.grid[1][0] + diag.dx[1] / 2., diag.grid[1][1] - diag.dx[1] / 2., diag.nx[1])
    z = np.linspace(diag.grid[2][0] + diag.dx[2] / 2., diag.grid[2][1] - diag.dx[2] / 2., diag.nx[2])

    # return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='cubic')
    return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='linear')

In [ ]:
def createSimDic(sim_labels, test):
    sim = {}
    for key in sim_labels.keys():
        sim[key] = {}
        for dtw in sim_labels[key]:
            sim[key][dtw] = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/{key}/dtw{dtw}/{key}.in")
    return sim

def p_parallel(sim, pusher, dtw, particle):
    B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,:]
    B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,:]
    B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,:]
    Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

    p1 = sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,:]
    p2 = sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,:]
    p3 = sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,:]

    p_par = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
    return p_par

def lin_fit(sim, quant, particle, titermin = 0, titermax = -1):

    t = sim["test_electrons"]["tracks"]["t"][particle, titermin:titermax]
    q = sim["test_electrons"]["tracks"][quant][particle, titermin:titermax]
    slope, intercept, r_value, p_value, std_err = linregress(t, q)
    
    return slope, intercept, r_value, p_value, std_err


In [ ]:
# Normalize axis to w_ce

def _set_scaled_formatter(axis, scale_factor, fmt=".2f"):
    axis.set_major_formatter(
        FuncFormatter(lambda v, pos: f"{v*scale_factor:{fmt}}")
    )

def scale_x_ax(scale_factor, fig, ax, label=r"$t[2\pi / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_xlabel(label)
    if lock_ticks:
        # freeze current tick positions
        ticks = ax.get_xticks()
        ax.xaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    return fig, ax

def scale_y_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_ylabel(label)
    if lock_ticks:
        ticks = ax.get_yticks()
        ax.yaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    return fig, ax

def scale_z_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_zlabel(label)
    if lock_ticks:
        ticks = ax.get_zticks()
        ax.zaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax

def scale_3d_axes(scale_factor, fig, ax, fmt=".2f"):
    ax.set_xlabel(r"$x_1[c / \Omega_e]$")
    ax.set_ylabel(r"$x_2[c / \Omega_e]$")
    ax.set_zlabel(r"$x_3[c / \Omega_e]$")
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, avrg_B, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0] * avrg_B / 2.0 / np.pi) + r"$ [2\pi / \Omega_e]$")
        label2 = ("$t = {:.3f}$".format(t[-1] * avrg_B / 2.0 / np.pi)  + r"$[ 2\pi / \Omega_e]$")


    if label == "Baseline":
        color = "black"
        markersize = 0
        linewidth = 0.6
    else:
        color = None
        markersize = 1.5
        linewidth = 0.8
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax)
    ax.legend()

    return fig, ax

In [ ]:
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/MS/RAW/electrons/RAW-electrons-000003.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/file_tags.tags", type="random", n_tags=5)

## Gyromotion 3d

In [ ]:
test = "Gyro"
dtw = "0_1"

sim_labels = {
    'Boris': ["1000", "100", "10", "1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, tmax=1, fig=fig, ax=ax)
for pusher in sim.keys():
    dtw = "10"
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=1, fig=fig, ax=ax)

# ax.view_init(elev=15, azim=80)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=10)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 20  

plt.show()

## Relativistic Gyromotion

In [ ]:
test = "GyroRel"
dtw = "0_1"

sim_labels = {
    'Boris': ["1000", "100", "10", "1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, tmax=1, fig=fig, ax=ax)
for pusher in sim.keys():
    dtw = "10"
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=1, fig=fig, ax=ax)

# ax.view_init(elev=0, azim=80)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=10)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 20  

plt.show()

## EXB 3d

In [ ]:
test = "EXB"
dtw = "0_1"

sim_labels = {
    'Boris': ["1000", "100", "10", "1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.0

# x3 / x1
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,index_min:index], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p3 / t
fig, ax = plt.subplots()
for pusher in ["Boris"]:
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p3", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
                label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# # p1 / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v1 / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# dtw="10"
# ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:] * 0.0 + 400./1000., 
#         label = "Theoretical", 
#         marker='o', linestyle='-', markersize=0.0, linewidth=0.6, color='black')

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v1 / rel error
fig, ax = plt.subplots()
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle)
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs(slope - slope_baseline) / slope_baseline)
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )


ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$dt \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_1$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# 3D Trajectory
fig, ax = None, None
for pusher in sim.keys():
    dtw = "10"
    index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=0.6, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, tmax=0.6, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()


## Relativistic $E\times B$ drift

In [ ]:
test = "EXBRel"
dtw = "0_1"

sim_labels = {
    'Boris': ["1000", "100", "10", "1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.0

# x3 / x1
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,index_min:index], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p3 / t
fig, ax = plt.subplots()
for pusher in ["Boris"]:
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p3", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
                label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# # p1 / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v1 / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# dtw="10"
# ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:] * 0.0 + 400./1000., 
#         label = "Theoretical", 
#         marker='o', linestyle='-', markersize=0.0, linewidth=0.6, color='black')

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v1 / rel error
fig, ax = plt.subplots()
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle)
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index_min = 1
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs(slope - slope_baseline) / slope_baseline)
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )


ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$dt \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_1$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# 3D Trajectory
fig, ax = None, None
for pusher in sim.keys():
    dtw = "10"
    index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), -1)
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index], fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()


## $\nabla B$ Drift

In [ ]:
sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, "GradB")


In [ ]:
particle = 0
avrg_B = 10.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2. / np.pi, fig=fig, ax=ax, fmt=".1f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        index_min = 1
        index = -1

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
                label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
ax.set_ylabel(r"$p_1[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        index_min = 1
        index = -1

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0. + slope, 
                label = fr"{pusher}, fit, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
ax.set_ylabel(r"$v_1[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    dtw = "100"
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim['Boris']["1"], "Baseline", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()

## Curvature Drift

In [ ]:
sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, "Curv")


In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2. / np.pi, fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    dtw = "1000"
    fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim['Boris']["1"], "Baseline", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()

## $(V_E\cdot\nabla )B$ drifts

In [ ]:
sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, "EGradB")


In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
                label = fr"{pusher}, x_1, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
                label = fr"{pusher}, x_2, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, x_3, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,1:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    for dtw in ["100", "1000"]:
        fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim['Boris']["1"], "Baseline", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()

## Mirror

In [ ]:
sim_labels = {
    # 'Boris': ["1000", "100", "10"],
    'Gca': ["1000", "100", "10"],
    # 'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, "Mirror")


In [ ]:
tmin = 7
tmax = 100000000000

sim_gca = sim["Gca"]["1000"]

gradB=get_gradB(sim_gca)
interp_gradB_1 = diag_interpolator(gradB[0], t=0)
interp_gradB_2 = diag_interpolator(gradB[1], t=0)
interp_gradB_3 = diag_interpolator(gradB[2], t=0)

J_b = get_Jacobian_b(sim_gca)
J_b_Interp = get_Jacobian_b_Interp(J_b, t=0)
fields = call_silently(get_fields, sim_gca, particle, tmin, tmax)
traj = call_silently(get_trajectory, sim_gca, particle, tmin, tmax)
p = call_silently(get_p, sim_gca, particle, tmin, tmax)

E = fields[0:3, :]
B = fields[3:6, :]
B_norm = np.linalg.norm(B, axis=0)
b = B / B_norm
b_B = b / B_norm
t = fields[-1, :]
dt = t[1]-t[0]

gradB_traj = np.stack([
    interp_gradB_1(traj[0:3].T),
    interp_gradB_2(traj[0:3].T),
    interp_gradB_3(traj[0:3].T)
], axis=0)

bGradB_traj = np.einsum('it,it->t', b, gradB_traj)


J_b_traj = np.empty((3, 3, np.shape(t)[0]), dtype=float)
for i in range(3):
    for j in range(3):
        J_b_traj[i,j] = J_b_Interp[i,j](traj[0:3].T)

curvature = np.einsum('tj,tij->it', b.T, J_b_traj.transpose(2, 0, 1))

# plt.plot(t, B_norm)

L_stop = (np.max(B_norm) - B_norm[2]) / np.max(bGradB_traj)

dtp = dt * np.sqrt(p[1, 2]**2 + p[2, 2]**2 + p[0, 2]**2)
print("dt p//0 = ", dtp, "L_stop =", L_stop)
    
# plt.plot(t, 1./bGradB_traj)
# plt.title("1 / (b . grad B)")
# plt.show()

plt.plot(t, (np.max(B_norm)-B_norm[2])/bGradB_traj)
plt.title("Delta B / (b . grad B)")
plt.show()

# plt.plot(t, B_norm/bGradB_traj)
# plt.title("B / (b . grad B)")
# plt.show()

# plt.plot(t, B_norm/np.linalg.norm(gradB_traj, axis=0))
# plt.title("B / (grad B)")
# plt.show()

# plt.plot(t, 1.0/np.linalg.norm(curvature, axis=0))
# plt.title("1 / (b . grad) b")
# plt.show()

plt.plot(t, dt * np.sqrt(p[1]**2 + p[2]**2 + p[0]**2))
plt.title("dt p")
plt.show()

print(dt)

In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in ["100", "1000"]:
        B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,2:]
        B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,2:]
        B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,2:]
        Bmag = np.sqrt(B1**2 + B2**2 + B3**2)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:] * B1/Bmag + 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:] * B2/Bmag +
                sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:] * B3/Bmag,
                label = fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
        # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
        #         np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
        #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
        #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
        #         label = fr"{pusher}, p, $dt\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
        #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
        # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
        #         sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:], 
        #         label = fr"{pusher}, $ p_1, dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
        #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
        #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:], 
        #         label = fr"{pusher}, $ p_2, dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
        #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
        #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
        #         label = fr"{pusher}, $ p_3, dt\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
        #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
# --- B on its own axis (right), same color as its line ---
ax2 = ax.twinx()

tB  = sim["Gca"]["100"]["test_electrons"]["tracks"]["t"][particle,2:]
B1  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B1"][particle,2:]
B2  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B2"][particle,2:]
B3  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B3"][particle,2:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

lnB, = ax2.plot(tB, Bmag, label=r"$B$", marker='o', linestyle='-',
                markersize=1.5, linewidth=0.8, color='purple')

cB = lnB.get_color()
ax2.spines["right"].set_color(cB)
ax2.tick_params(axis='y', colors=cB)
ax2.yaxis.label.set_color(cB)
ax2.set_ylabel(r"$B$")

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax2.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_\parallel[c]$")

# legend including B
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax.legend(h1 + h2, l1 + l2, loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid()

plt.show()



p_par_100 = p_parallel(sim, "Gca", "100", particle)
p_par_1000 = p_parallel(sim, "Gca", "1000", particle)

fig, ax = plt.subplots()
ax.plot(sim["Gca"]["1000"]["test_electrons"]["tracks"]["t"][particle,:], np.abs(p_par_100[::10]-p_par_1000))
ax.set_ylabel(r"$|p_{||, 100} - p_{||, 1000}| [c]$")
fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    for dtw in [ "100", "1000"]:
        fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $dt\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim['Boris']["1"], "Baseline", particle=particle, avrg_B=avrg_B, tmax=5000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

plt.show()

In [ ]:
def UtoV(u):
    return u / np.sqrt(1 + u**2)

def VtoU(v):
    return v * np.sqrt(1 - v**2)

def UtoGamma(u):
    return np.sqrt(1 + u**2)


u = np.sqrt(0.7**2 + 0.7**2 + 0.7**2)
print("u = ", u, "\t\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.5**2 + 0.5**2 + 0.1**2)
print("u = ", u, "\t\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.1**2 + 0.1**2 + 0.05**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.05**2 + 0.05**2 + 0.001**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.05**2 + 0.7**2 + 0.7**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

np.sqrt(0.7**2 + 0.7**2)